# Image registration using homography

In [ ]:
# Import 2 images
# Convert to grey scale
# Initiate ORB detector
# Find key points and describe them
# Match keypoints - Brute force matcher
# RANSAC (reject bad keypoints)
# Regiter two images (use homology)

import cv2
import numpy as np
im1 = cv2. imread ('images/monkey_distorted.jpg') #Imag to be registered
im2 = cv2. imread (' images/monkey.jpg')

#Reference image
img1 = cv2.cvtColor (im1, cv2. COLOR_BGR2GRAY)
img2 = cv2.cvtColor (im2, cv2.COLOR_BGR2GRAY)

#Initiate ORB
orb = cv2.ORB_create (50)
kp1, des1 = orb.detectAndCompute (img1, None)
kp2, des2 = orb.detectAndCompute (img2, None)
matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)

#Match descriptors
matches = matcher.match(des1, des2, None)

matches = sorted(matches, key = lambda x:x.distance)
points1 = np.zeros((len(matches), 2), dtype=np.float32)
points2 = np.zeros((len(matches), 2), dtype=np.float32)

for i, match in enumerate(matches) :
    points1[i, :] = kp1[match.queryIdx].pt
    points2[i, :] = kp2[match.trainIdx].pt

h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

# Use homography
height, width, channels = im2. shape

imReg = cv2.warpPerspective(im1, h, (width, height))
img3 = cv2.drawMatches(im1, kp1, im2, kp2, matches [:10], None)
cv2.imshow("Registered image", imReg)
cv2.waitKey(0)

# Grain size analysis

In [ ]:
# Step 1: Read image and define pixel size (if needed to convert results into microns, not p
# Step 2: Denoising, if required and threshold image to separate grains from boundaries.
# Step 3: Clean up image, if needed (erode, etc.) and create a mask for grains
# Step 4: Label grains in the masked image
# Step 5: Measure the properties of each grain (object)
# Step 6: Output results into a csv file

import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage
from skimage import io, color, measure

# Step 1
img = cv2.imread("images/denoising/noise.jpg")
pixel_to_um = 0.5 # 1 pixel = 0.5 um or 500 nm

# Step 2
median_blur = cv2.medianBlur(img, 3)
ret, thresh = cv2.threshold(median_blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Step 3
kernel = np.ones ((3,3), np.uint8)
eroded = cv2.erode(th, kernel, iterations=1)
dilated = cv2.dilate(eroded, kernel, iterations=2)
mask = dilated == 255

# Erosion: It shrinks the bright regions (foreground) in the binary image. 
#          It's useful for removing small white noise or breaking narrow connections in the foreground.
# Dilation: It expands the bright regions in the binary image. 
#           It's useful for closing small gaps in the foreground or connecting broken parts of objects.

# Step 4
s = [[1,1,1], [1,1,1], [1,1,1]]
labeled_mask, num_labels = ndimage. label (mask, structure=s)

img2 = color. label2gb(labeled_mask, bg_label=0)

# Step 5
clusters = measure.regionprops(labeled_mask, img)

#Step 6
propList = ['Area',
            'equivalent_diameter',
            'orientation',
            'ManorAxistength',
            'Perimeter',
            'MinIntensity',
            'MaxIntensity',
            'MeanIntensity']


output_file = open('image_measurements.csv', 'w')
output_file.write((',' + ",".join(propList) + '\n'))

for cluster_props in clusters:
    # output cluster properties to the excel file
    output_file.write(str[cluster_props['Label']])
    for i, prop in enumerate(propList):
        if(prop == 'Area'):
            to_print = cluster_props[prop]*pixel_to_um**2 #convert pixel square to um square
        elif(prop == 'orientation'):
            to_print = cluster_props[prop]*57.2958 #Convert to degress from radians 
        elif(prop.find('Intensity')<0):
            to_print = cluster_props[prop]*pixel_to_um #Convert porps, basically the ones with
        else:
            to_print = cluster_props[prop] #Reamining props, basically the ones with 
        output_file.write(',' + str(to_print))
    output_file.write('\n')